# Install dependencies

In [ ]:
# nano-graphrag is cloned in the external/ directory
# ran this to install nano-graphrag in editable mode
# pip install -e external/nano-graphrag

In [ ]:
# run this before the rest of the notebook to install ollama and its embedding model
brew install ollama
brew services start ollama

ollama --version
ollama pull nomic-embed-text

In [ ]:
!pip install ollama

## Helper Functions

### 1. File Ingestion 

In [28]:
import gzip
import json
from pathlib import Path

DATA_DIR = "./"  

def load_json_gz(path):
    with gzip.open(path, "rt", encoding="utf-8") as f:
        return json.load(f)

### 2. Data cleanup 

- we probably need this but did not start yet
- would need to fix insert function after adding this

## Nano-graphRAG Implementation

- minimum modification of the ollama example code
- need to add things after data cleansing in the insert function 
- and add some prompt engineering things to the query
- would also need to look into the source code constructing graphs and whatnot to achieve Milstones 3 and 4
- also I think we need a lighter version of this lol 

In [30]:
import os
import sys

import nest_asyncio
nest_asyncio.apply()

sys.path.append("..")
import logging
import ollama
import numpy as np
from nano_graphrag import GraphRAG, QueryParam
from nano_graphrag.base import BaseKVStorage
from nano_graphrag._utils import compute_args_hash, wrap_embedding_func_with_attrs

logging.basicConfig(level=logging.WARNING)
logging.getLogger("nano-graphrag").setLevel(logging.INFO)

# Assumed llm model settings
MODEL = "llama3"

# Assumed embedding model settings
EMBEDDING_MODEL = "nomic-embed-text"
EMBEDDING_MODEL_DIM = 768
EMBEDDING_MODEL_MAX_TOKENS = 8192


async def ollama_model_if_cache(
    prompt, system_prompt=None, history_messages=[], **kwargs
) -> str:
    # remove kwargs that are not supported by ollama
    kwargs.pop("max_tokens", None)
    kwargs.pop("response_format", None)

    ollama_client = ollama.AsyncClient()
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    # Get the cached response if having-------------------
    hashing_kv: BaseKVStorage = kwargs.pop("hashing_kv", None)
    messages.extend(history_messages)
    messages.append({"role": "user", "content": prompt})
    if hashing_kv is not None:
        args_hash = compute_args_hash(MODEL, messages)
        if_cache_return = await hashing_kv.get_by_id(args_hash)
        if if_cache_return is not None:
            return if_cache_return["return"]
    # -----------------------------------------------------
    response = await ollama_client.chat(model=MODEL, messages=messages, **kwargs)

    result = response["message"]["content"]
    # Cache the response if having-------------------
    if hashing_kv is not None:
        await hashing_kv.upsert({args_hash: {"return": result, "model": MODEL}})
    # -----------------------------------------------------
    return result


def remove_if_exist(file):
    if os.path.exists(file):
        os.remove(file)


WORKING_DIR = "./nano_graphrag_cache_ollama_TEST"

def query():
    rag = GraphRAG(
        working_dir=WORKING_DIR,
        best_model_func=ollama_model_if_cache,
        cheap_model_func=ollama_model_if_cache,
        embedding_func=ollama_embedding,
    )

    q = "Summarize the main AI course policies."  # ← CHANGE THIS
    result = rag.query(q, param=QueryParam(mode="global"))

    print(f"query: {q}\nanswer: {result}")
    

def insert():
    from time import time

    # clear previous index/cache
    remove_if_exist(f"{WORKING_DIR}/vdb_entities.json")
    remove_if_exist(f"{WORKING_DIR}/kv_store_full_docs.json")
    remove_if_exist(f"{WORKING_DIR}/kv_store_text_chunks.json")
    remove_if_exist(f"{WORKING_DIR}/kv_store_community_reports.json")
    remove_if_exist(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")

    rag = GraphRAG(
        working_dir=WORKING_DIR,
        enable_llm_cache=True,
        best_model_func=ollama_model_if_cache,
        cheap_model_func=ollama_model_if_cache,
        embedding_func=ollama_embedding,
    )

    start = time()

    # load all your json.gz files
    for file in Path(DATA_DIR).glob("*.json.gz"):
        with gzip.open(file, "rt", encoding="utf-8") as f:
            obj = json.load(f)

        if isinstance(obj, list):
            for i, item in enumerate(obj):
                text = item.get("text", "").strip()
                if text:
                    print(f"Inserting text from file: {file}, item {i} | length: {len(text)}")
                    print("Preview:", text[:200])
                    rag.insert(text)
                else:
                    print(f"Skipped item {i} in file {file}: no text")
        elif isinstance(obj, dict):
            text = obj.get("text", "").strip()
            if text:
                print(f"Inserting text from file: {file} | length: {len(text)}")
                print("Preview:", text[:200])
                rag.insert(text)
            else:
                print(f"Skipped file {file}: no text")
        else:
            print(f"Skipped file {file}: not a dict or list")

    print("indexing time:", time() - start)

    # ------------------- INSPECTION -------------------
    # print("\n--- Inspect Chunks ---")
    # chunks_file = Path(WORKING_DIR) / "kv_store_text_chunks.json"
    # with open(chunks_file, "r", encoding="utf-8") as f:
    #     chunks = json.load(f)
    # print("Total chunks:", len(chunks))
    # for k, v in list(chunks.items())[:3]:
    #     print(k)
    #     print(v.get("text")[:200])
    #     print("---")

    # print("\n--- Inspect Full Documents ---")
    # full_docs_file = Path(WORKING_DIR) / "kv_store_full_docs.json"
    # with open(full_docs_file, "r", encoding="utf-8") as f:
    #     full_docs = json.load(f)
    # print("Total documents:", len(full_docs))
    # for doc_id, doc in list(full_docs.items())[:2]:
    #     print(doc_id)
    #     print(doc.get("text")[:200])
    #     print("---")

    # print("\n--- Inspect Graph ---")
    # graph_file = Path(WORKING_DIR) / "graph_chunk_entity_relation.graphml"
    # import networkx as nx
    # G = nx.read_graphml(graph_file)
    # print("Number of nodes:", G.number_of_nodes())
    # print("Number of edges:", G.number_of_edges())



# We're using Ollama to generate embeddings for the BGE model
@wrap_embedding_func_with_attrs(
    embedding_dim=EMBEDDING_MODEL_DIM,
    max_token_size=EMBEDDING_MODEL_MAX_TOKENS,
)
async def ollama_embedding(texts: list[str]) -> np.ndarray:
    embed_text = []
    for text in texts:
        data = ollama.embeddings(model=EMBEDDING_MODEL, prompt=text)
        embed_text.append(data["embedding"])

    return embed_text


if __name__ == "__main__":
    insert()
    query()


INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-4o'
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 0 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './nano_graphrag_cache_ollama_TEST/vdb_entities.json'} 0 data
INFO:nano-graphrag:[New Docs] inserting 1 docs
INFO:nano-graphrag:[New Chunks] inserting 1 chunks
INFO:nano-graphrag:[Entity Extraction]...


Inserting text from file: scraped_pages.json.gz, item 0 | length: 1848
Preview: What this course is all about Artificial Intelligence (AI) addresses one of the ultimate puzzles humans are trying to solve: How is it possible for a brain, whether biological or electronic, to percei


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


INFO:nano-graphrag:Inserting 3 vectors to entities


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:nano-graphrag:[Community Report]...
/Users/jinheeshim/.pyenv/versions/3.12.3/lib/python3.12/inspect.py:3045: RuntimeWarning: coroutine 'GraphRAG.aquery' was never awaited
  params = OrderedDict((param.name, param) for param in parameters)
INFO:nano-graphrag:Each level has communities: {0: 1}
INFO:nano-graphrag:Generating by levels: [0]
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


...

⠇ Processed 38 communities


INFO:nano-graphrag:Writing graph with 1323 nodes, 542 edges
INFO:nano-graphrag:[New Docs] inserting 1 docs


Inserting text from file: scraped_pages.json.gz, item 42 | length: 99429
Preview: /*! * Bootstrap Icons v1.13.1 (https://icons.getbootstrap.com/) * Copyright 2019-2024 The Bootstrap Authors * Licensed under MIT (https://github.com/twbs/icons/blob/main/LICENSE) */ @font-face { font-


INFO:nano-graphrag:[New Chunks] inserting 31 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


KeyboardInterrupt: 